In [ ]:
import pandas as pd
import numpy as np
import io
import re
import csv
from spacy.lang.id import Indonesian
from itertools import zip_longest
nlp_id_spacy = Indonesian()
pos_term = []
pos_term_neg = []
csv.register_dialect('myDialect', delimiter=',')

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

barasa_path = r'drive/My Drive/Dataset/barasa.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# lokasi untuk file barasa sebagai sumber lexical pada tugas akhir
df = pd.read_csv(barasa_path, delimiter='\t', index_col=False)

# file_input sebagai variabel sumber file tweet yang telah didapat dari proses preprocessing
file_input = "indosatcare-labelled.txt"
file = open(file_input, mode='r', encoding='utf-8')
text = file.readlines()

# variabel file untuk menyimpan hasil proses sentiwordnet. Ganti nama file jika dibutuhkan
file = "data_DIC_INDOSAT.csv"
senti_dic = open(file, mode='w')

In [ ]:
# fungsi untuk memanggil kamus kelas kata (POS Tagging Bahasa Indonesia). ganti file sesuai dengan lokasi kamus kata
def loadPos_id(file = 'kata_dasar.txt'):
    kata_pos = {}
    df=open(file,"r",encoding="utf-8", errors='replace')
    data=df.readlines();df.close()
    
    for line in data:
        d = line.split()
        kata = d[0].strip()
        
        pos = d[-1].strip().replace("(",'').replace(')','')
        kata_pos[kata] = pos
        
    return kata_pos

In [ ]:
text

['ter regist ringtone padahal tidak me regist sampah s,0\n',
 'quota mksdnya gimana,0\n',
 'min cek dm tolong,0\n',
 'paket internet hilang ya,0\n',
 'malam kak maaf ketidaknyamanannya lapor konfirmasi via dm ya salam teo,1\n',
 'hai kak maaf bgt ya kak yuk coba setting manual jaring ya cara pilih atur jari,1\n',
 'provider lambat canda lambat mahalnyadoang,0\n',
 'sya udh kali coba beli paket kuota hot promo freedom you 10gb gagal transaksi s,0\n',
 'cantik hi kak coba setting jaring manual pilih operator pilih indosat sukses,1\n',
 'tak kiro muk wkwk makane saiki busak trus browser,0\n',
 'min tolong dititidak yaa kuota 50gb tp gabisa akses internet tolong dititidak sec,0\n',
 'jika kakak kenan kakak konfirmasi infoin data2 via dm,1\n',
 'hi kak maaf keluh belum bantu titidaklanjuti sms tsb bantu bl,1\n',
 'min ane transfer pulsa drtd gagal nih,0\n',
 'halo nomer indosat udah tidak aktif tp kartu bisa diaktifin tidak min,0\n',
 'spam every day,0\n',
 'hai kak konfirmasi via dm ya yuk

In [ ]:
print(df[df['lemma'].isnull()])

Empty DataFrame
Columns: [synset, language, goodness, lemma, PosScore, NegScore]
Index: []


In [ ]:
# fungsi untuk memberikan kelas kata pada setiap kata tweet yang telah didapat
kata_pos = loadPos_id()
pos_term.clear()

for idx, texts in enumerate(text):
    tweet = texts.split(",")
    print(tweet)
    label = tweet[1].strip('\n')
    print(label)
    wordList = tweet[0].split()
    
    for term in wordList:
        try:
            pos_term.append([idx, term , kata_pos[term], label[0]]) 
        except:
            pos_term.append([idx, term , 'n', label[0]]) 

['ter regist ringtone padahal tidak me regist sampah s', '0\n']
0
['quota mksdnya gimana', '0\n']
0
['min cek dm tolong', '0\n']
0
['paket internet hilang ya', '0\n']
0
['malam kak maaf ketidaknyamanannya lapor konfirmasi via dm ya salam teo', '1\n']
1
['hai kak maaf bgt ya kak yuk coba setting manual jaring ya cara pilih atur jari', '1\n']
1
['provider lambat canda lambat mahalnyadoang', '0\n']
0
['sya udh kali coba beli paket kuota hot promo freedom you 10gb gagal transaksi s', '0\n']
0
['cantik hi kak coba setting jaring manual pilih operator pilih indosat sukses', '1\n']
1
['tak kiro muk wkwk makane saiki busak trus browser', '0\n']
0
['min tolong dititidak yaa kuota 50gb tp gabisa akses internet tolong dititidak sec', '0\n']
0
['jika kakak kenan kakak konfirmasi infoin data2 via dm', '1\n']
1
['hi kak maaf keluh belum bantu titidaklanjuti sms tsb bantu bl', '1\n']
1
['min ane transfer pulsa drtd gagal nih', '0\n']
0
['halo nomer indosat udah tidak aktif tp kartu bisa diaktifin tid

In [ ]:
pos_term

[[0, 'ter', 'n', '0'],
 [0, 'regist', 'n', '0'],
 [0, 'ringtone', 'n', '0'],
 [0, 'padahal', 'l', '0'],
 [0, 'tidak', 'adv', '0'],
 [0, 'me', 'n', '0'],
 [0, 'regist', 'n', '0'],
 [0, 'sampah', 'n', '0'],
 [0, 's', 'n', '0'],
 [1, 'quota', 'n', '0'],
 [1, 'mksdnya', 'n', '0'],
 [1, 'gimana', 'n', '0'],
 [2, 'min', 'adj', '0'],
 [2, 'cek', 'n', '0'],
 [2, 'dm', 'n', '0'],
 [2, 'tolong', 'v', '0'],
 [3, 'paket', 'n', '0'],
 [3, 'internet', 'n', '0'],
 [3, 'hilang', 'v', '0'],
 [3, 'ya', 'l', '0'],
 [4, 'malam', 'n', '1'],
 [4, 'kak', 'n', '1'],
 [4, 'maaf', 'n', '1'],
 [4, 'ketidaknyamanannya', 'n', '1'],
 [4, 'lapor', 'v', '1'],
 [4, 'konfirmasi', 'n', '1'],
 [4, 'via', 'l', '1'],
 [4, 'dm', 'n', '1'],
 [4, 'ya', 'l', '1'],
 [4, 'salam', 'n', '1'],
 [4, 'teo', 'n', '1'],
 [5, 'hai', 'i', '1'],
 [5, 'kak', 'n', '1'],
 [5, 'maaf', 'n', '1'],
 [5, 'bgt', 'n', '1'],
 [5, 'ya', 'l', '1'],
 [5, 'kak', 'n', '1'],
 [5, 'yuk', 'n', '1'],
 [5, 'coba', 'v', '1'],
 [5, 'setting', 'n', '1'],
 [5, 'm

In [ ]:
# %%time
d = []
q = []
tweet_id = 'tweet_id'
term = 'term'
pos = 'pos'
pos_fre = 'pos_fre'
neg_fre = 'neg_fre'
label = 'label'
q.append(tweet_id)
q.append(term)
q.append(pos)
q.append(pos_fre)
q.append(neg_fre)
q.append(label)

# file csv untuk menyimpan hasil sentiwordnet 
csv_save = csv.writer(senti_dic,dialect='myDialect')
csv_save.writerow(q)

for itter in range(len(pos_term)):
    count_positif = 0
    count_negatif = 0

    item = df[df['lemma']==pos_term[itter][1]]
    if pos_term[itter][2] == "adj":
        pos_synset = "a"
    elif pos_term[itter][2] == "adv":
        pos_synset = "adv"
    elif pos_term[itter][2] == "n":
        pos_synset = "n"
    elif pos_term[itter][2] == "v":
        pos_synset = "v"
    elif pos_term[itter][2] == "l":
        pos_synset = "l"
    elif pos_term[itter][2] == "k":
        pos_synset = "r"
        
    item = item[item['language'].str.contains("I|B")]
    item = item[item['synset'].str.contains(pos_synset)]
    item = item[item['goodness'].str.contains("Y|O|M")]
    item = item[item['PosScore']-item['NegScore'] != 0]
    
    item_POS = item[item['PosScore']-item['NegScore'] > 0]
    item_NEG = item[item['PosScore']-item['NegScore'] < 0]

    if len(item.index) > 0:
        if len(item_POS) > 0:
            count_positif += len(item_POS)   
        if len(item_NEG) > 0:
            count_negatif += len(item_NEG)
        
        # save csv
        d.append(pos_term[itter][0])
        d.append(pos_term[itter][1])
        d.append(pos_term[itter][2])
        d.append(count_positif)
        d.append(count_negatif)
        d.append(pos_term[itter][3])
        
        csv_save = csv.writer(senti_dic,dialect='myDialect')
        csv_save.writerow(d)
        
    d.clear()   
    
senti_dic.close()

In [ ]:
pos_term

[[0, 'ter', 'n', '0'],
 [0, 'regist', 'n', '0'],
 [0, 'ringtone', 'n', '0'],
 [0, 'padahal', 'l', '0'],
 [0, 'tidak', 'adv', '0'],
 [0, 'me', 'n', '0'],
 [0, 'regist', 'n', '0'],
 [0, 'sampah', 'n', '0'],
 [0, 's', 'n', '0'],
 [1, 'quota', 'n', '0'],
 [1, 'mksdnya', 'n', '0'],
 [1, 'gimana', 'n', '0'],
 [2, 'min', 'adj', '0'],
 [2, 'cek', 'n', '0'],
 [2, 'dm', 'n', '0'],
 [2, 'tolong', 'v', '0'],
 [3, 'paket', 'n', '0'],
 [3, 'internet', 'n', '0'],
 [3, 'hilang', 'v', '0'],
 [3, 'ya', 'l', '0'],
 [4, 'malam', 'n', '1'],
 [4, 'kak', 'n', '1'],
 [4, 'maaf', 'n', '1'],
 [4, 'ketidaknyamanannya', 'n', '1'],
 [4, 'lapor', 'v', '1'],
 [4, 'konfirmasi', 'n', '1'],
 [4, 'via', 'l', '1'],
 [4, 'dm', 'n', '1'],
 [4, 'ya', 'l', '1'],
 [4, 'salam', 'n', '1'],
 [4, 'teo', 'n', '1'],
 [5, 'hai', 'i', '1'],
 [5, 'kak', 'n', '1'],
 [5, 'maaf', 'n', '1'],
 [5, 'bgt', 'n', '1'],
 [5, 'ya', 'l', '1'],
 [5, 'kak', 'n', '1'],
 [5, 'yuk', 'n', '1'],
 [5, 'coba', 'v', '1'],
 [5, 'setting', 'n', '1'],
 [5, 'm